# Salary Model Version 1
## Author: Xiaochi (George) Li

Input: "processed_data.pickle" from the preprocessing part  
Spec: No missing value, target variable "salary" should be the last column


In [1]:
import pandas as pd

In [7]:
df = pd.read_pickle("processed_data.pickle")
print("shape:",df.shape)
df.head()

shape: (80000, 59)


,Total Payments,Lump Sum Pay,Other Pay (Payroll Explorer),FMS Department,Job Class,Average Benefit Cost,p_dep,Rate,p_bonus,t_bonus,...,p_grade_L,p_grade_M,p_grade_N,p_grade_S,p_grade_T,p_grade_V,p_grade_X,p_grade_Y,p_grade_Z,Salary
161542,4847.83,0.0,1740.73,98,941,18755.63,4594.945881,31.29,0.00,0.0,...,0,0,0,0,0,0,0,0,0,3107.10
167941,56481.08,0.0,3480.12,14,1358,11435.16,1603.000000,26.47,2869.48,0.0,...,0,0,0,0,0,0,0,0,0,51560.00
244088,138165.59,0.0,4609.98,98,3115,19392.62,4594.945881,38.26,0.00,0.0,...,0,0,0,0,0,0,0,0,0,78058.00
15392,111777.71,0.0,14526.10,70,2214,12740.52,4301.000000,52.66,13293.10,1233.0,...,0,0,0,0,0,0,0,0,0,94630.06
69682,9961.87,0.0,565.20,88,2498,0.00,7903.000000,14.79,0.00,304.0,...,0,0,0,0,0,0,0,0,0,9396.67


In [9]:
X = df[df.columns[:-1]]
y = df[df.columns[-1]]
X.shape

(80000, 58)